## Data Preprocessing

In [1]:
# loading libraries
import librosa   #for audio processing
import librosa.display
import wave
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import sys
from scipy.io import wavfile #for audio processing
import warnings
warnings.filterwarnings("ignore")

In [2]:
# loading scripts
sys.path.insert(1, '../scripts')
sys.path.append("..")
sys.path.append(".")

from data_cleaning import DataCleaner

DC = DataCleaner("../logs/preprocessing_notebook.log")

In [3]:
# loading meta data
path = "../data/train"

meta_data = DC.generate_metadata(path)
meta_data.head()


,Target,Feature
0,ያንደኛ ደረጃ ትምህርታቸው ን ጐንደር ተ ም ረዋል,../data/train/wav/tr_1_tr01001.wav
1,የተ ለቀቁት ምርኮኞች በ አካባቢያቸው ሰላማዊ ኑሮ እንዲ ኖሩ የ ትራንስፖ...,../data/train/wav/tr_2_tr01002.wav
2,በ አዲስ አበባው ስታዲየም በ ተካሄዱ ት ሁለት ግጥሚያ ዎች በ መጀመሪያ ...,../data/train/wav/tr_3_tr01003.wav
3,ወሬው ን ወሬ ያደረጉ ምስጢረ ኞች ናቸው,../data/train/wav/tr_4_tr01004.wav
4,ኢትዮጵያዊ ቷ በ ብሄራዊ ባህላዊ አለባበስ ከ አለም አንደኝነት ን ተቀዳጀ ች,../data/train/wav/tr_5_tr01005.wav


In [16]:
meta_data.loc[0,"Feature"]

'../data/train/wav/tr_1_tr01001.wav'

In [25]:
# number of channels
def channel_count(df):
    n_list=[]
    for i in range(df.shape[0]):
        try:
            data = wave.open(df.loc[i,"Feature"], mode = 'rb')
        except:
            n_list.append(200)
            continue
        
        channel = data.getparams().nchannels
        n_list.append(channel)
    df["n_channel"] = n_list

    return df





In [26]:
new_df = channel_count(meta_data)
len(new_df)

1000